**MLSP Project**

Check gpu spec

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14979172305904329877
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16154099712
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9161889133773819455
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

Copy and decompress the raw wav files

In [ ]:
!mkdir wav
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav1.zip ./wav1.zip 
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav2.zip ./wav2.zip ; unzip -qq "./wav1.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav3.zip ./wav3.zip ; unzip -qq "./wav2.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav4.zip ./wav4.zip ; unzip -qq "./wav3.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav5.zip ./wav5.zip ; unzip -qq "./wav4.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav6.zip ./wav6.zip ; unzip -qq "./wav5.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav7.zip ./wav7.zip ; unzip -qq "./wav6.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav8.zip ./wav8.zip ; unzip -qq "./wav7.zip" -d "./wav"
!cp ./drive/MyDrive/mlsp_project/dataset/wavzip/wav9.zip ./wav9.zip ; unzip -qq "./wav8.zip" -d "./wav"
!unzip -qq "./wav9.zip" -d "./wav"

cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav1.zip': No such file or directory
cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav2.zip': No such file or directory
unzip:  cannot find or open ./wav1.zip, ./wav1.zip.zip or ./wav1.zip.ZIP.
cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav3.zip': No such file or directory
unzip:  cannot find or open ./wav2.zip, ./wav2.zip.zip or ./wav2.zip.ZIP.
cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav4.zip': No such file or directory
unzip:  cannot find or open ./wav3.zip, ./wav3.zip.zip or ./wav3.zip.ZIP.
cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav5.zip': No such file or directory
unzip:  cannot find or open ./wav4.zip, ./wav4.zip.zip or ./wav4.zip.ZIP.
cp: cannot stat './drive/MyDrive/mlsp_project/dataset/wavzip/wav6.zip': No such file or directory
unzip:  cannot find or open ./wav5.zip, ./wav5.zip.zip or ./wav5.zip.ZIP.
cp: cannot stat './drive/MyDrive/mlsp_proj

audio preprocess function

In [ ]:
from scipy.io import wavfile
from scipy.signal import lfilter, stft
import numpy as np

#reference: https://github.com/Derpimort/VGGVox-PyTorch
#https://github.com/a-nagrani/VGGVox
def preprocess(audio, is_train):
    fs = 16000
    frame_len = int(20 * fs / 1000)
    hop_len = int(10 * fs / 1000)

    nfft = 512

    audio = lfilter([1, -1],[1,-0.99],audio)
    # dither add noise
    dither = np.random.uniform(low=-1, high=1, size=audio.shape)
    std_dev = np.std(audio)
    audio = audio + (1e-6 * std_dev) * dither

    # preemphasis
    audio = lfilter([1, -0.97], 1, audio)

    _, _, spectrogram = stft(
        audio,
        fs=fs,
        window=np.hamming(frame_len),
        nperseg=frame_len,
        noverlap=frame_len-hop_len,
        nfft=nfft,
        return_onesided=False,
        boundary=None,
        padded=False
        )
    spectrogram = np.abs(spectrogram)
    zero_mean_spectrogram = spectrogram-spectrogram.mean(1, keepdims=True)
    spectrogram_std_dev = np.clip(spectrogram.std(1, keepdims=True), 1e-12, None)
    spectrogram = zero_mean_spectrogram/spectrogram_std_dev

    #put code below here if use the truncated test spectrogram

    return spectrogram

In [ ]:
if not is_train:
  spect_len = 0
  for i in range(1100,100,-100):
      if spectrogram.shape[1] >= i:
          spect_len = i
          break
  start = (spectrogram.shape[1]-spect_len)//2
  return spectrogram[:,start:spect_len+start]

loader

In [ ]:
import torch
from torch.utils.data import Dataset

data_path = './wav/'
split_path = './drive/MyDrive/mlsp_project/iden_split.txt'

train_size = 145265
test_size = 8251
audio_len = 49000

class VoxCeleb1_Spectrogram(Dataset):
    def __init__(self, is_train):
        f = open(split_path, "r")
        lines = f.readlines()
        f.close()
        self.is_train = is_train
        self.ids = []
        self.data = []
        self.paths = []
        if is_train:
            self.dataset_size = train_size
        else:
            self.dataset_size = test_size
        counter = 0
        for line in lines:
            if counter == self.dataset_size:
                break
            strings = line.split(' ')
            set_label = int(strings[0])
            if is_train:
                if set_label == 1 or set_label == 2:
                    self.ids.append(int(strings[1].split('/')[0].replace('id',''))-10001)
                    path = data_path + strings[1].replace('\n', '')
                    #self.paths.append(path)
                    fs, audio = wavfile.read(path)
                    self.data.append(audio)
                    counter += 1
            else:
                if set_label == 3:
                    self.ids.append(int(strings[1].split('/')[0].replace('id',''))-10001)
                    path = data_path + strings[1].replace('\n', '')
                    #self.paths.append(path)
                    fs, audio = wavfile.read(path)
                    self.data.append(audio)
                    counter += 1

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        #fs, audio = wavfile.read(self.paths[idx])
        audio = self.data[idx]
        #for train set, randomly select a selected length fragment of the audio each time
        #then pass to the preprocess function to get a fix length spectrogram
        #for test set, pass full length audio to the preprocess function
        #then generate a rounded length spectrogram for testing
        if self.is_train:
            start = np.random.randint(0, audio.shape[0] - audio_len + 1)
            audio = audio[start:start + audio_len]
        spectrogram = preprocess(audio,self.is_train)
        return torch.from_numpy(spectrogram).unsqueeze(0), self.ids[idx]

In [ ]:
batch_size = 100
trainset = VoxCeleb1_Spectrogram(is_train=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
batch_size = 1
testset = VoxCeleb1_Spectrogram(is_train=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

Network

In [ ]:
import torch
import torchvision
import torch.nn as nn

net = torchvision.models.resnet34()
net.conv1  = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
net.fc = nn.Linear(512, 1251)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net.to(device)

#from torchsummary import summary

#summary(net.to(device), (1, 512, 300))

cuda:0


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Optimizer

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)

Load 

In [ ]:
state_dict = torch.load('drive/MyDrive/mlsp_project/backup/temp/Resnet34_temp.pth') 
net.load_state_dict(state_dict)
state_dict1 = torch.load('drive/MyDrive/mlsp_project/backup/temp/Resnet34Optimizer_temp.pth')
optimizer.load_state_dict(state_dict1)
state_dict2 = torch.load('drive/MyDrive/mlsp_project/backup/temp/Resnet34Scheduler_temp.pth')
print(state_dict2)
scheduler.load_state_dict(state_dict2)

{'gamma': 0.97, 'base_lrs': [0.001], 'last_epoch': 55, '_step_count': 56, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [0.00018726015763786058]}


Train

In [ ]:
print(torch.cuda.get_device_properties(0).total_memory/(1024**3))
print(torch.cuda.memory_reserved(0)/(1024**3))
print(torch.cuda.memory_allocated(0)/(1024**3))
import gc
del inputs, labels
gc.collect()
print(torch.cuda.get_device_properties(0).total_memory/(1024**3))
print(torch.cuda.memory_reserved(0)/(1024**3))
print(torch.cuda.memory_allocated(0)/(1024**3))

15.8992919921875
14.728515625
14.631492137908936
15.8992919921875
14.728515625
0.08260107040405273


In [ ]:
for epoch in range(1, 100):
  #train
  net.train()
  train_loss = 0
  train_correct = 0
  train_batch_num = len(trainloader)
  for batch_idx, (inputs, labels) in enumerate(trainloader, 0):
    inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_correct += (outputs.argmax(dim=1) == labels).int().sum()
  scheduler.step()
  print('\nEpoch: {}\t Train: Loss: {:.7f}\tAccuracy: {}/{}\t'.format(\
      epoch, train_loss/train_batch_num, train_correct, 145265))
  
  torch.save(net.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34_temp.pth')
  torch.save(optimizer.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34Optimizer_temp.pth')
  torch.save(scheduler.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34Scheduler_temp.pth')
  #backup every 5 epoch
  if epoch % 5 == 0:
    net.eval()
    epoch_test_loss = 0
    test_correct = 0
    test_batch_num = len(testloader)
    with torch.no_grad():
      for batch_idx, (inputs, labels) in enumerate(testloader, 0):
        inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        epoch_test_loss += loss.item()
        test_correct += (outputs.argmax(dim=1) == labels).int().sum()
    print('Test: Loss: {:.7f}\tAccuracy: {}/{}\t'.format(\
        epoch_test_loss/test_batch_num, test_correct, test_size))
    
    torch.save(net.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/Resnet34_file{}.pth'.format(str(epoch//5)))
    torch.save(optimizer.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/Resnet34Optimizer_file{}.pth'.format(str(epoch//5)))
    torch.save(scheduler.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/Resnet34Scheduler_file{}.pth'.format(str(epoch//5)))
  
  



Epoch: 1	 Train: Loss: 0.4386139	Accuracy: 131840/145265	

Epoch: 2	 Train: Loss: 0.4306985	Accuracy: 132302/145265	

Epoch: 3	 Train: Loss: 0.4254959	Accuracy: 132457/145265	

Epoch: 4	 Train: Loss: 0.4202378	Accuracy: 132758/145265	

Epoch: 5	 Train: Loss: 0.4141645	Accuracy: 132899/145265	
Test: Loss: 0.9593897	Accuracy: 6331/8251	

Epoch: 6	 Train: Loss: 0.4075999	Accuracy: 133222/145265	

Epoch: 7	 Train: Loss: 0.4024958	Accuracy: 133412/145265	

Epoch: 8	 Train: Loss: 0.3975529	Accuracy: 133637/145265	

Epoch: 9	 Train: Loss: 0.3931192	Accuracy: 133766/145265	

Epoch: 10	 Train: Loss: 0.3859891	Accuracy: 134172/145265	
Test: Loss: 0.9671407	Accuracy: 6344/8251	

Epoch: 11	 Train: Loss: 0.3788208	Accuracy: 134459/145265	

Epoch: 12	 Train: Loss: 0.3790125	Accuracy: 134435/145265	

Epoch: 13	 Train: Loss: 0.3716156	Accuracy: 134777/145265	

Epoch: 14	 Train: Loss: 0.3667277	Accuracy: 134920/145265	

Epoch: 15	 Train: Loss: 0.3622567	Accuracy: 135100/145265	
Test: Loss: 0.9200108	A

KeyboardInterrupt: ignored

save

In [ ]:
torch.save(net.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34.pth')
torch.save(optimizer.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34Optimizer.pth')
torch.save(scheduler.state_dict(), \
               'drive/MyDrive/mlsp_project/backup/temp/Resnet34Scheduler.pth')

Test with full length Spectrogram instead of 100 to 1000

In [ ]:
state_dict = torch.load('drive/MyDrive/mlsp_project/model/Resnet34_spectro.pth') 
net.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
for i in range(1, 6):
  net.eval()
  epoch_test_loss = 0
  test_correct = 0
  test_batch_num = len(testloader)
  with torch.no_grad():
    for batch_idx, (inputs, labels) in enumerate(testloader, 0):
      inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
      outputs = net(inputs)
      loss = criterion(outputs, labels)

      epoch_test_loss += loss.item()
      test_correct += (outputs.argmax(dim=1) == labels).int().sum()
  print('Test: Loss: {:.7f}\tAccuracy: {}/{}\t ({:.6f})'.format(\
      epoch_test_loss/test_batch_num, test_correct, test_size, test_correct/test_size*100))

Test: Loss: 0.7008811	Accuracy: 6934/8251	 (84.038300)
Test: Loss: 0.7008811	Accuracy: 6934/8251	 (84.038300)
Test: Loss: 0.7008820	Accuracy: 6934/8251	 (84.038300)
Test: Loss: 0.7008814	Accuracy: 6934/8251	 (84.038300)
Test: Loss: 0.7008814	Accuracy: 6934/8251	 (84.038300)
